In [8]:
import pandas as pd

data = {
    'YEAR' :    [2018,2018,2017,2018,2018,2018],
    'SEASON':   ['SPRING', 'SPRING', 'WINTER', 'SPRING', 'SPRING', 'SPRING'],
    'CODE':     ['A', 'A', 'A', 'B', 'C', 'D'],
    'BUDGET':   [500,200,300,4000,700,0],
    'QUANTITY': [1000,1000,1000,2000,300,4000]
}

df = pd.DataFrame(data)
df

,YEAR,SEASON,CODE,BUDGET,QUANTITY
0,2018,SPRING,A,500,1000
1,2018,SPRING,A,200,1000
2,2017,WINTER,A,300,1000
3,2018,SPRING,B,4000,2000
4,2018,SPRING,C,700,300
5,2018,SPRING,D,0,4000


In [11]:
f = {
        'CODE' : 'nunique', 
        'BUDGET' : 'sum'
}

g = df.groupby(['YEAR', 'SEASON']).agg({'BUDGET' : 'sum', "QUANTITY":"sum"})

g



,,BUDGET,QUANTITY
YEAR,SEASON,,
2017,WINTER,300,1000
2018,SPRING,5400,8300


In [10]:

v1 = g
v1

,,BUDGET,QUANTITY
YEAR,SEASON,,
2017,WINTER,300,1000
2018,SPRING,5400,7300


In [6]:
v2 = g.agg(lambda x: x.drop_duplicates('CODE', keep='first').QUANTITY.sum())
v2

YEAR  SEASON
2017  WINTER    1000
2018  SPRING    7300
dtype: int64

In [7]:

df = pd.concat([v1, v2.to_frame('QUANTITY')], 1)

df

,,CODE,BUDGET,QUANTITY
YEAR,SEASON,,,
2017,WINTER,1,300,1000
2018,SPRING,1,5400,7300


# References

1. Pandas Groupby: Summarising, Aggregating, and Grouping data in Python by Shane Lynn
https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/ 

In [12]:
df.columns

Index(['YEAR', 'SEASON', 'CODE', 'BUDGET', 'QUANTITY'], dtype='object')

In [ ]:
import pandas as pd

# You can access the data with the following commands :
customers_df = pd.read_csv('customer_data.csv')
orders_df = pd.read_csv('orders_data.csv')

def compute_relative_sales_difference():
    # Write your code here
    orders_df["transaction_year"] = pd.to_datetime(orders_df["transaction_date"]).dt.year
    
    
    quantity_2017 = orders_df[orders_df["transaction_year"] == 2017]["quantity"]
    price_2017 = orders_df[orders_df["transaction_year"] == 2017]["price"]
    sales_2017 = sum(quantity_2017*price_2017)
    
    quantity_2018 = orders_df[orders_df["transaction_year"] == 2018]["quantity"]
    price_2018 = orders_df[orders_df["transaction_year"] == 2018]["price"]
    sales_2018 = sum(quantity_2018*price_2018)
    
    return sales_2018/sales_2017 -1
    

In [ ]:

import pandas as pd
import datetime

customers_df = pd.read_csv('customer_data.csv')
orders_df = pd.read_csv('orders_data.csv')

def get_median_age_per_acquisition_channel():
    # Write your code here
    
    
    customers_df["age"] = 2021 - pd.to_datetime(customers_df["birth_date"]).dt.year
    
    
    acq_df = customers_df.groupby(["acquisition_channel"]).agg(
        median_age = pd.NamedAgg(column="age",aggfunc="median")
    )
    acq_df = acq_df.reset_index()

    # This is the expected format for the output (you can delete this line)
    return acq_df

In [ ]:
import pandas as pd

customers_df = pd.read_csv('customer_data.csv')
orders_df = pd.read_csv('orders_data.csv')


def get_most_popular_product():
    # Write your code here

    customers_df["birth_year"] = pd.to_datetime(customers_df["birth_date"]).dt.year
    
    millennial_df = customers_df[(customers_df["birth_year"] >= 1981) & 
                                 (customers_df["birth_year"] <= 1996)]

    millennial_id = millennial_df["customer_id"]
    
    mil_order_df = orders_df[orders_df["customer_id"].isin(millennial_id)]
                           
    product_df  = mil_order_df.groupby(["product"]).agg(
        total_sold = pd.NamedAgg(column="quantity",aggfunc="sum")
    )
    product_df = product_df.reset_index()     

    most_popular_product = product_df.loc[product_df["total_sold"].idxmax(),"product"]
                                 
    return most_popular_product
                                 

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression

customers_df = pd.read_csv('customer_data.csv')
orders_df = pd.read_csv('orders_data.csv')

def main():
    # Write your code here
    # highest value customer, predict demand for a given customer and year
    
    customers_df["age"] = 2021 - pd.to_datetime(customers_df["birth_date"]).dt.year

    orders_df["year"] = pd.to_datetime(orders_df["transaction_date"]).dt.year
    orders_df["revenue"] = orders_df["quantity"]*orders_df["price"]
    
    revenus_df = orders_df.merge(customers_df,how="left",on="customer_id")
    
    rev_year_cid_df = revenus_df.groupby(["year","customer_id"]).agg(
        age=pd.NamedAgg(column="age",aggfunc="first"), # use first since they are the same
        total_revenue=pd.NamedAgg(column="revenue",aggfunc="sum"),
        purchase_count=pd.NamedAgg(column="revenue",aggfunc="count"),
        #prev_year_revenue=pd.NamedAgg(),
        #prev_year_nb_products=pd.NamedAgg(),
    )
    
    # I ran out of time to figure out how to write the two features using agg
    # I can write a python forloop to compute these values and store them back into the grouped df
    
    rev_year_cid_df = rev_year_cid_df.reset_index()
    
    prev_year_revenue, prev_year_nb_products = [],[]
    for row in rev_year_cid_df.iterrows():
        print(row[1]["year"])
        prev_rev = sum(revenus_df[(revenus_df["year"] == int(row[1]["year"])-1) & 
                                    (revenus_df["customer_id"] == row["customer_id"])])
                                    
        prev_year_revenue.append(prev_rev)                            
        
        break
    
    print(prev_year_revenue)
        
        
        
    # 
    
    print(rev_year_cid_df.head())
    
    training_set = rev_year_cid_df[rev_year_cid_df["year"]<2018]
    testing_set = rev_year_cid_df[rev_year_cid_df["year"]==2018]
    
    train_X,train_y = training_set["age",""]
    
    
    return 0
    
    